In [40]:
import sys
sys.path.append("./../src/")

In [41]:
import random
seed = 0
random.seed(seed)

import pandas as pd
import numpy as np

from utilities import DATA_PATH, RESPONSE_COL_NAME

In [42]:
# Read in preprocessed data
df = pd.read_csv(f"{DATA_PATH}/data.csv")
display(df)

,lapse_ape_grp_fe5fb8,stat_flag_LAPSED,is_Singapore,lapse_ape_grp_e91421,recency_hlthclaim_unsuccess,n_months_last_bought_32c74c,n_months_last_bought_grp_fe5fb8,flg_hlthclaim_14cb37_ever,race_desc_Indian,lapse_ape_lh_839f8a,...,n_months_last_bought_839f8a,f_mindef_mha,tot_inforce_pols,flg_latest_being_lapse,is_valid_dm,is_valid_email,giclaim_amt,n_months_since_lapse_grp_caa6ff,n_months_last_bought_gi,f_purchase_lh
0,0.0,0,1,0.0,False,9999.0,9999.0,1,0,0.0,...,9999.0,0,3.0,0,1.0,1.0,False,9999.0,9999.0,1
1,0.0,0,1,0.0,False,9999.0,9999.0,1,0,0.0,...,9999.0,0,1.0,0,1.0,0.0,False,9999.0,9999.0,1
2,0.0,0,1,0.0,False,9999.0,9999.0,1,0,0.0,...,9999.0,0,1.0,0,1.0,1.0,False,0.0,9999.0,1
3,0.0,0,1,0.0,False,9999.0,9999.0,1,0,0.0,...,9999.0,0,1.0,0,1.0,1.0,False,0.0,9999.0,1
4,0.0,0,1,0.0,False,9999.0,9999.0,1,0,0.0,...,9999.0,0,1.0,0,1.0,1.0,False,0.0,9999.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17979,0.0,1,1,0.0,False,36.0,9999.0,1,0,0.0,...,9999.0,0,1.0,0,0.0,1.0,False,9999.0,9999.0,1
17980,0.0,0,1,0.0,False,9999.0,9999.0,1,0,0.0,...,9999.0,0,1.0,0,1.0,1.0,False,0.0,9999.0,1
17981,0.0,0,1,0.0,False,9999.0,9999.0,1,0,0.0,...,9999.0,0,1.0,0,1.0,1.0,False,0.0,9999.0,1
17982,0.0,0,1,0.0,False,9999.0,9999.0,1,0,0.0,...,9999.0,1,2.0,1,0.0,1.0,False,9999.0,46.0,1


In [43]:
for c in df.columns:
    if df[c].nunique()==2 and df[c].dtype in ['int64','float64']:
        df[c] = df[c].astype('bool')
        print(df[c].value_counts())

False    17209
True       775
Name: stat_flag_LAPSED, dtype: int64
True     17781
False      203
Name: is_Singapore, dtype: int64
True     16609
False     1375
Name: flg_hlthclaim_14cb37_ever, dtype: int64
False    17135
True       849
Name: race_desc_Indian, dtype: int64
False    13673
True      4311
Name: is_consent_to_sms, dtype: int64
False    17300
True       684
Name: flg_substandard, dtype: int64
False    16970
True      1014
Name: is_dependent_in_at_least_1_policy, dtype: int64
False    13844
True      4140
Name: f_elx, dtype: int64
False    16287
True      1697
Name: race_desc_Others, dtype: int64
True     16599
False     1385
Name: is_sg_pr, dtype: int64
False    15961
True      2023
Name: is_consent_to_call, dtype: int64
False    12524
True      5460
Name: is_consent_to_mail, dtype: int64
False    13586
True      4398
Name: ape_grp, dtype: int64
False    17930
True        54
Name: is_housewife_retiree, dtype: int64
True     16752
False     1232
Name: f_ever_declined_la, dtyp

In [44]:
df[RESPONSE_COL_NAME].value_counts()

True     17280
False      704
Name: f_purchase_lh, dtype: int64

To split the data, we take 20% x 704 (140 samples) positive samples to the validation and test datasets each, as well as 140 negative samples. The rest go to the training dataset:

|          Train         |     Validation     |        Test        |
|------------------------|--------------------|--------------------|
|  424 positive samples  |140 positive samples|140 positive samples|
| 17000 negative samples |140 negative samples|140 negative samples|

In [45]:
n_train_positive, n_train_negative = 424, 17000
n_val_positive, n_val_negative = 140, 140
n_test_positive, n_test_negative = 140, 140

negative_indexes = np.where(df[RESPONSE_COL_NAME] == 1)[0].tolist()
positive_indexes = np.where(df[RESPONSE_COL_NAME] == 0)[0].tolist()

random.shuffle(negative_indexes)
random.shuffle(positive_indexes)

In [46]:
train_indices = positive_indexes[0 : n_train_positive] + negative_indexes[0 : n_train_negative]
val_indices = positive_indexes[n_train_positive : n_train_positive + n_val_positive] + negative_indexes[n_train_negative : n_train_negative + n_val_negative]
test_indices = positive_indexes[n_train_positive + n_val_positive : ] + negative_indexes[n_train_negative + n_val_negative : ]

train = df.iloc[train_indices, :].reset_index(drop=True)
val = df.iloc[val_indices, :].reset_index(drop=True)
test = df.iloc[test_indices, :].reset_index(drop=True)

In [47]:
train.to_csv(f"{DATA_PATH}/train.csv", index=False)
val.to_csv(f"{DATA_PATH}/val.csv", index=False)
test.to_csv(f"{DATA_PATH}/test.csv", index=False)

In [51]:
positive_data = df.iloc[positive_indexes, :].reset_index(drop=True)
positive_data.to_csv(f"{DATA_PATH}/positive.csv", index=False)